In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"프로젝트 루트를 sys.path에 추가했습니다: {project_root}")

프로젝트 루트를 sys.path에 추가했습니다: c:\Users\user\potenup\ReNe


In [4]:
from src.core.config import settings
import gspread
from langchain_openai import ChatOpenAI
from google.oauth2.service_account import Credentials
from datetime import datetime
from dotenv import load_dotenv

In [ ]:
os.getenv()
load_dotenv()

In [ ]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [6]:
SCOPES = [
    "https://www.googleapis.com/auth/drive",
]

In [ ]:
def read_transcript_from_path(file_path: str):
    """ 로컬 경로에서 텍스트 파일을 읽습니다. """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            return content
    except FileNotFoundError as e:
        print("해당 파일을 찾을 수 없습니다: {e}")
        return None
    except Exception as e:
        print("파일을 로드 중 오류가 발생했습니다: {e}")
        return None
    



In [ ]:

def summarize_with_openai(text_to_summarize):
    """ GPT를 이용해 회의 텍스트를 요약합니다. """

    system_prompt = (
        
    )

In [ ]:
def write_to_google_sheet(meeting_title, summary_text):
    """(3) Google Sheets에 최종 결과를 기록합니다."""